In [1]:
!pip install opencv-python

In [2]:
import numpy as np
import pandas as pd
import torch
import os
import cv2
import matplotlib.pyplot as plt
import torch.nn.functional as F 
from sklearn.model_selection import train_test_split
from torch import nn,optim
from torchvision import transforms,datasets,models
from torchvision.utils import make_grid
from torch.utils.data import DataLoader, ConcatDataset
from collections import OrderedDict
from tqdm import tqdm
pd.options.plotting.backend = "plotly"
from torch import nn, optim
import time
import copy
from datetime import datetime
import torch.backends.cudnn as cudnn
from torch.optim import lr_scheduler

In [3]:
train_transforms = transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

test_transforms = transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

transformer = {
    'dataset1': transforms.Compose([transforms.Resize(255),
                                            transforms.CenterCrop(224),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.RandomRotation(10),
                                            transforms.RandomGrayscale(),
                                            transforms.RandomAffine(translate=(0.05,0.05), degrees=0),
                                            transforms.ToTensor()
                                           ]),
    
    'dataset2' : transforms.Compose([transforms.Resize(255),
                                            transforms.CenterCrop(224),
                                            transforms.RandomHorizontalFlip(p=1),
                                            transforms.RandomGrayscale(),
                                            transforms.RandomAffine(translate=(0.1,0.05), degrees=10),
                                            transforms.ToTensor()
                                    
                                           ]),
    'dataset3' : transforms.Compose([transforms.Resize(255),
                                            transforms.CenterCrop(224),
                                            transforms.RandomHorizontalFlip(p=0.5),
                                            transforms.RandomRotation(15),
                                            transforms.RandomGrayscale(p=1),
                                            transforms.RandomAffine(translate=(0.08,0.1), degrees=15),
                                            transforms.ToTensor()
                                           ]),
}

In [4]:
path = "./chest_xray/chest_xray"

dataset1 = datasets.ImageFolder(path+'/train', 
                      transform=transformer['dataset1'])

dataset2 = datasets.ImageFolder(path+'/train', 
                      transform=transformer['dataset2'])

dataset3 = datasets.ImageFolder(path+'/train', 
                      transform=transformer['dataset3'])

test_size1 = int(3875/(1341+3875) * len(dataset2))
train_size1 = len(dataset2) - test_size1

test_size2 = int(4023/(1341+3875) * len(dataset3))
train_size2 = len(dataset3) - test_size2

norm1, _ = torch.utils.data.random_split(dataset2, [train_size1, test_size1])
norm2, _ = torch.utils.data.random_split(dataset3, [train_size2, test_size2])

print(len(norm1))
print(len(norm2))

# norm1, _ = train_test_split(dataset2, test_size= 3875/(1341+3875), shuffle=False)
# norm2, _ = train_test_split(dataset3, test_size= 4023/(1341+3875), shuffle=False)

# print(len(norm1))
# print(len(norm2))

dataset = ConcatDataset([dataset1, norm1, norm2])
trainset = dataset

#trainset = datasets.ImageFolder(os.path.join(path, 'train'),transform = train_transforms)
testset = datasets.ImageFolder(os.path.join(path, 'test'),transform = test_transforms)

1341
1193


In [5]:
#class_names = trainset.classes
#print(class_names)
# print(trainset.class_to_idx)
print(testset.classes)

['NORMAL', 'PNEUMONIA']


In [6]:
print(len(trainset))
print(len(testset))

# train_set_size = 4000
# valid_set_size = len(trainset) - train_set_size
# trainset, valid_set = torch.utils.data.random_split(trainset, [train_set_size, valid_set_size])

# test_set_size = 600
# valid_set_size = len(testset) - test_set_size
# testset, valid_set = torch.utils.data.random_split(testset, [test_set_size, valid_set_size])

# print(len(trainset))
# print(len(testset))

batch_size = 50

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle= True, num_workers=4, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size= 50,
                                         shuffle= False, num_workers=4, pin_memory=True)
dataiter = iter(trainloader)
images, labels = next(dataiter)

7750
624


In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, best_acc = 0.0):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        model.train()  # Set model to training mode

        running_loss = 0.0
        running_corrects = 0
        counter = 0
        # Iterate over data.
        for inputs, labels in trainloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            counter += 1
            if counter % 100 == 99:    # print every 100 mini-batches
              now = datetime.now()
              current_time = now.strftime("%H:%M:%S")
              print("Current Time =", current_time)
              print(f"Epoch: {epoch} Seen:{(counter+1)*10}")

        scheduler.step()

        epoch_loss = running_loss / len(trainloader.dataset)
        epoch_acc = running_corrects.double() / len(trainloader.dataset)

        print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


        model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0
        counter = 0
        # Iterate over data.
        print("~~~~Testing~~~~")
        for inputs, labels in testloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            counter += 1
            if counter % 100 == 99:    # print every 100 mini-batches
              now = datetime.now()
              current_time = now.strftime("%H:%M:%S")
              print("Current Time =", current_time)
              print(f"Epoch: {epoch} Seen:{(counter+1)*10}")

        epoch_loss = running_loss / len(testloader.dataset)
        epoch_acc = running_corrects.double() / len(testloader.dataset)

        print(f'Testing Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # deep copy the model
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [8]:
cudnn.benchmark = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#model_ft = models.vgg16_bn()

model_ft = models.densenet161(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False
    
in_features = model_ft.classifier.in_features

model_ft.classifier = nn.Linear(in_features, 2)

# model_ft = models.resnet18(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

C:\Users\Stan\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Stan\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10, best_acc = 0.0)

Epoch 0/9
----------
Current Time = 17:40:33
Epoch: 0 Seen:1000
Training Loss: 0.2809 Acc: 0.8881
~~~~Testing~~~~
Testing Loss: 0.2870 Acc: 0.8798

Epoch 1/9
----------
Current Time = 17:42:12
Epoch: 1 Seen:1000
Training Loss: 0.1625 Acc: 0.9453
~~~~Testing~~~~
Testing Loss: 0.2834 Acc: 0.8846

Epoch 2/9
----------
Current Time = 17:43:44
Epoch: 2 Seen:1000
Training Loss: 0.1423 Acc: 0.9486
~~~~Testing~~~~
Testing Loss: 0.2898 Acc: 0.8862

Epoch 3/9
----------
Current Time = 17:45:20
Epoch: 3 Seen:1000
Training Loss: 0.1331 Acc: 0.9535
~~~~Testing~~~~
Testing Loss: 0.2968 Acc: 0.8862

Epoch 4/9
----------
Current Time = 17:46:57
Epoch: 4 Seen:1000
Training Loss: 0.1356 Acc: 0.9526
~~~~Testing~~~~
Testing Loss: 0.2901 Acc: 0.8894

Epoch 5/9
----------
Current Time = 17:48:30
Epoch: 5 Seen:1000
Training Loss: 0.1348 Acc: 0.9529
~~~~Testing~~~~
Testing Loss: 0.3067 Acc: 0.8846

Epoch 6/9
----------
Current Time = 17:50:03
Epoch: 6 Seen:1000
Training Loss: 0.1334 Acc: 0.9523
~~~~Testing~~~

In [10]:
save_path = './testrresnetmymodel.pt'
torch.save(model_ft.state_dict(), save_path)

In [11]:
# save_path = './testresnetmymodel.pt'
# net = Net()
# net.load_state_dict(torch.load(save_path))  
# net.eval()

In [12]:
testimages = []
testlabels = []
for image, label in testset:
    testimages.append(image.numpy())
    testlabels.append(label)
    
#print(testimages[0])
testimages = np.array(testimages, dtype=np.float32)
#print(testimages[0])

In [13]:
from torch.utils.data import Dataset
class AttackDataset(Dataset):
    def __init__(self, attack_images, attack_labels,  transforms=None):
        self.attack_images = attack_images
        self.attack_labels = attack_labels
        self.transforms = transforms
        self.data_len = len(attack_images)

    def __len__(self):
        return self.data_len
  
    def __getitem__(self, idx):
        image = self.attack_images[idx]
        #print(np.shape(image))
        label = self.attack_labels[idx]
        if self.transforms:
            image = self.transforms(image)
        return (image, label)

In [14]:
print("starting")

converted_testimages = []

for i in range(0, len(testimages)):
    #print(np.shape(testimages[i]))
    convert = testimages[i].transpose(1,2,0)
    #print(np.shape(convert))
    converted_testimages.append(convert)
    
    
test_transforms2 = transforms.Compose([
        transforms.ToTensor(),
    ])

attack_trainset = ConcatDataset([trainset, AttackDataset(converted_testimages,testlabels, test_transforms2)])
batch_size = 10
print("end")
attack_trainloader = torch.utils.data.DataLoader(attack_trainset, batch_size=batch_size,
                                          shuffle= True, num_workers=0)
print("start2")
dataiter = iter(attack_trainloader)
images, labels = next(dataiter)
print("end2")
for i in range(1):
  #print(images[i].shape, labels[i])
    transform = transforms.ToPILImage()
    img = transform(images[i+2])
    img.show()

starting
end
start2
end2
